# What My Goals Are

## Input Data

While the scraper is running, I want to find what my goals are for this project. Since I've stuck to using Earnings Calls, the similarity ratings arent all that interesting - we can find details about the companies and classify them without using NLP at all - they're literally categorized into sectors. 

### Analyst Opinions

These are also not analyst opinions. In the future, I would absolutely want to include analyst opinions, however, for V1 of this project it doesnt make sense. To get a good overview, I'd want multiple different opinions from multiple different sources - I have a list to look at, but these are different websites, and we'd need a different scraper for each one. So thats stressful:) Additionally, sentiments would be the analysts opinion of the company rather than what the earnings said - so it would be more for processing the analysts 

## My Problems

### The calls themselves 
I've tried listening to calls - they're so boring. Everyone tries very hard to be very polite and slippery - "Im not saying anything and Im sure you have a good explanation and its probably a good thing, but it seems youre reporting a lost of billions of dollars, could you please clarify how thats a good thing" 

Theyre exhausting to read and exhausting to listen to. Summarize, please.

### Big Trends? 
There are a lot of companies. I know whats happening with the big tech ones because I've worked near them - but what about the industry? 
Can we find common topics between companies that aren't part of the company definitions - how companies are approaching offices, outsourcing, etc. 

## Steps 

### Topic Analysis 

### Summarizing 
Not easily covered. 


In [4]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import re
import os 
import sys
import time
import pandas as pd

from IPython.core.debugger import Pdb
from random import Random, randrange

from sklearn.feature_extraction.text import CountVectorizer
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from pysentiment2 import LM 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
#open text file in read mode
text_file = open('all_documents_1000.txt', "r")
 
#read whole file to a string
raw_string = text_file.read()
 
#close file
text_file.close()
raw_docs = raw_string.split('\n********************************************************************\n')[1:]

In [7]:
title_file = open('all_titles_1000.txt', 'r')
raw_titles= title_file.read()
title_file.close()
titles = raw_titles.split('\n********************************************************************\n')[1:]


In [8]:
def doc_clean(thing):
    doc = BeautifulSoup(thing, 'html.parser')
    a = doc.prettify()
    a = a.replace(u'\xa0', u' ')
    a = a.replace(u'&amp;',u'and')
    return a


def split_by_sections(doc):
    x = doc.split('<h2>\n')
    details_by_block = {}
    for block in x:
        
        block_2 = block.split('</h2>\n')
        try:
            block_text = block_2[1]
            block_title = block_2[0]
        except:
            block_text = block_2[0]
            block_title = 'header'
        
        details_by_block[block_title]= block_text
    return details_by_block

In [9]:
df_big = {titles[i]:doc_clean(raw_docs[i]) for i in range(0,len(raw_docs))}
docs = list(df_big.values())
doc = docs[0]

In [10]:
print(split_by_sections(doc).keys())
print(split_by_sections(docs[1]).keys())

dict_keys(['header', '  Contents:\n ', '  Prepared Remarks:\n ', '  Questions and Answers:\n ', '  Call participants:\n '])
dict_keys(['header', '  Contents:\n ', '  Prepared Remarks:\n ', '  Questions and Answers:\n ', '  Call participants:\n '])


In [11]:
sections_split = { titles[i]: split_by_sections(docs[i]) for i in range(0, len(docs)) }
sections_split_list = list(sections_split.values())
df = pd.DataFrame(sections_split_list)

In [12]:
df.columns = ['header','toc','prepared_remarks','questions_and_answers','who']
df.head()

,header,toc,prepared_remarks,questions_and_answers,who
0,"<span class=""article-content"">\n <p>\n <stron...",<ul>\n <li>\n Prepared Remarks\n </li>\n ...,<p>\n <strong>\n Operator\n </strong>\n <...,<p>\n <strong>\n Operator\n </strong>\n <...,<p>\n <strong>\n Douglas C. Yearley\n </s...
1,"<span class=""article-content"">\n <p>\n <stron...",<ul>\n <li>\n Prepared Remarks\n </li>\n ...,"<br data-uw-rm-sr="""" role=""presentation""/>\n ...","<br data-uw-rm-sr="""" role=""presentation""/>\n ...",<p>\n <strong>\n Megan Britt\n </strong>\...
2,"<span class=""article-content"">\n <p>\n <stron...",<ul>\n <li>\n Prepared Remarks\n </li>\n ...,<br/>\n <p>\n <strong>\n Operator\n </str...,<br/>\n <p>\n <strong>\n Operator\n </str...,<p>\n <strong>\n Whitney Notaro\n </stron...
3,"<span class=""article-content"">\n <p>\n <stron...",<ul>\n <li>\n Prepared Remarks\n </li>\n ...,"<br data-uw-rm-sr="""" role=""presentation""/>\n ...","<br data-uw-rm-sr="""" role=""presentation""/>\n ...",<p>\n <strong>\n Vince Klinges\n </strong...
4,"<span class=""article-content"">\n <p>\n <stron...",<ul>\n <li>\n Prepared Remarks\n </li>\n ...,"<br data-uw-rm-sr="""" role=""presentation""/>\n ...","<br data-uw-rm-sr="""" role=""presentation""/>\n ...",<p>\n <strong>\n David Atchley\n </strong...


In [19]:
def clean_prepared(pp):
    p = BeautifulSoup(pp, 'html.parser').text
    p = p.replace(u'\n\n', u' ')
    p = p.replace(u'\n', u'')
    try:
        p = p.split('    --     ', 1)[1]
    except:
        p=p
    try:
        p = p.split('     ', 1)[1]
    except:
        p=p
        
    return p

#Lets start with the prepared remarks
p = [clean_prepared(p) for p in list(df['prepared_remarks'])]


In [ ]:
tokens = pos_tag(word_tokenize(p[0]))
entities = ne_chunk(tokens)

In [23]:
analyzer = SentimentIntensityAnalyzer()
score = analyzer.polarity_scores(p[500])
score

{'neg': 0.018, 'neu': 0.845, 'pos': 0.138, 'compound': 0.9999}